<a href="https://colab.research.google.com/github/kerplunkykz431/kaggle_sleep/blob/main/02_%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E7%A2%BA%E8%AA%8D%E3%81%A8%E5%89%8D%E5%87%A6%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.データの準備

In [14]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<img src="https://i.gyazo.com/02758e3141e46a4d2d681cb125c5dbd0.png" width="662"/>

In [15]:
%%bash
# ドライブ経由だと処理が遅くなるため、inputデータをランタイム配下にコピーします
# (エラーが発生する場合があります。その場合はセルを再実行してください)
INPUT_PATH="/content/drive/MyDrive/2023_nishika_sleep/input"
cp -r ${INPUT_PATH} /content/   

In [16]:
# ファイルの解凍(少し時間がかかります)
!tar -zxf ./input/edf_data.tar.gz -C ./input/

In [17]:
# 脳波データ（EDF形式）を扱う専用ライブラリのインストール
# 参考：https://mne.tools/stable/index.html
!pip install -q mne==1.2.0

In [18]:
import datetime
from pathlib import Path
from typing import Dict, List

import numpy as np
import pandas as pd
import mne
from tqdm.auto import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [19]:
# パスの設定
INPUT_DIR = Path("./input/")
EDF_DIR = INPUT_DIR / "edf_data"

# 1.sample submissionの確認
予測結果を提出するCSVのフォーマットを確認します

In [20]:
sample_submission_df = pd.read_csv(INPUT_DIR/"sample_submission.csv", parse_dates=[1])

In [21]:
sample_submission_df

,id,meas_time,condition
0,53c1555,1989-11-20 23:19:30,Sleep stage W
1,53c1555,1989-11-20 23:20:00,Sleep stage W
2,53c1555,1989-11-20 23:20:30,Sleep stage W
3,53c1555,1989-11-20 23:21:00,Sleep stage W
4,53c1555,1989-11-20 23:21:30,Sleep stage W
...,...,...,...
52291,9b444bb,1989-04-12 07:32:30,Sleep stage W
52292,9b444bb,1989-04-12 07:33:00,Sleep stage W
52293,9b444bb,1989-04-12 07:33:30,Sleep stage W
52294,9b444bb,1989-04-12 07:34:00,Sleep stage W


meas_timeは睡眠段階の予測時間です。  
睡眠段階は30秒毎に判断します。そのため、30秒毎に行があるのが分かります。

# 2.被験者メタデータの確認


In [22]:
train_record_df = pd.read_csv(INPUT_DIR/"train_records.csv")
test_record_df = pd.read_csv(INPUT_DIR/"test_records.csv")

In [23]:
train_record_df.head(5)

,id,subject_id,night,age,sex,lights_off,psg,hypnogram
0,3c1c5cf,07c46da,1,90,male,23:28:00,3c1c5cf-PSG.edf,3c1c5cf-Hypnogram.edf
1,8fbd71b,07c46da,2,90,male,01:29:00,8fbd71b-PSG.edf,8fbd71b-Hypnogram.edf
2,9d5e9ec,21969ff,1,51,female,23:10:00,9d5e9ec-PSG.edf,9d5e9ec-Hypnogram.edf
3,e0df8c0,21969ff,2,51,female,23:15:00,e0df8c0-PSG.edf,e0df8c0-Hypnogram.edf
4,3e404fc,22b58e8,1,51,female,22:38:00,3e404fc-PSG.edf,3e404fc-Hypnogram.edf


各カラムの説明は以下となります

* id:行に一意のID
* subject_id:被験者にユニークなID
* night:測定した夜のID
* sex:性別
* hypnogram:アノテーションデータのファイル名
* psg:センサーデータのファイル名

In [24]:
test_record_df.head(3)

,id,subject_id,night,age,sex,lights_off,psg
0,53c1555,17ca2cd,1,91,female,00:15:00,53c1555-PSG.edf
1,29ef1d5,17ca2cd,2,91,female,23:39:00,29ef1d5-PSG.edf
2,c90b6e7,2c77159,1,56,female,23:55:00,c90b6e7-PSG.edf


テストにいる患者には、ラベルデータである`hypnogram`のカラムが存在しないことが分かる

年齢のヒストグラム 

trainは50代が多いことが分かる

In [25]:
go.Figure(
    data=[
        go.Histogram(
            x=train_record_df.groupby("subject_id").agg(age=("age", "first"))["age"],
            opacity=0.75,
            name='train',
        ),
        go.Histogram(
            x=test_record_df.groupby("subject_id").agg(age=("age", "first"))["age"],
            opacity=0.75,
            name='test',
        )
    ],
    layout=dict(barmode='overlay', width=800),
)

性別の割合 

testは女性が多いことが分かる

In [26]:
# train
train_record_df.groupby("subject_id").agg(sex=("sex", "first")).value_counts() / train_record_df["subject_id"].nunique()

sex   
male      0.545455
female    0.454545
dtype: float64

In [27]:
# test
test_record_df.groupby("subject_id").agg(sex=("sex", "first")).value_counts() / test_record_df["subject_id"].nunique()

sex   
female    0.695652
male      0.304348
dtype: float64

患者のメタ情報の分布はtrainとtestで異なることが分かりました。   
睡眠の質は年齢等で異なる可能性がありそうなので、このあたりの考慮も必要かもしれない。

trainとtestで患者の重複.  

In [28]:
set(train_record_df["subject_id"]) & set(test_record_df["subject_id"])

set()


重複はありません。  
モデルの性能を測る検証セットはなるべくtestを模倣するのが定石です。  
検証セットを作成する際は、患者で検証データを分けた方が良さそう

In [29]:
# ファイルパスを設定
train_record_df["hypnogram"] = train_record_df["hypnogram"].map(lambda x: str(EDF_DIR/x))
train_record_df["psg"] = train_record_df["psg"].map(lambda x: str(EDF_DIR/x))
test_record_df["psg"] = test_record_df["psg"].map(lambda x: str(EDF_DIR/x))

In [30]:
train_record_df.head(5)

,id,subject_id,night,age,sex,lights_off,psg,hypnogram
0,3c1c5cf,07c46da,1,90,male,23:28:00,input/edf_data/3c1c5cf-PSG.edf,input/edf_data/3c1c5cf-Hypnogram.edf
1,8fbd71b,07c46da,2,90,male,01:29:00,input/edf_data/8fbd71b-PSG.edf,input/edf_data/8fbd71b-Hypnogram.edf
2,9d5e9ec,21969ff,1,51,female,23:10:00,input/edf_data/9d5e9ec-PSG.edf,input/edf_data/9d5e9ec-Hypnogram.edf
3,e0df8c0,21969ff,2,51,female,23:15:00,input/edf_data/e0df8c0-PSG.edf,input/edf_data/e0df8c0-Hypnogram.edf
4,3e404fc,22b58e8,1,51,female,22:38:00,input/edf_data/3e404fc-PSG.edf,input/edf_data/3e404fc-Hypnogram.edf


# 3.被験者1名のデータの表示
被験者(subject)のある晩(night)のデータを見てみましょう。  


In [31]:
row = train_record_df.iloc[0]
row

id                                         3c1c5cf
subject_id                                 07c46da
night                                            1
age                                             90
sex                                           male
lights_off                                23:28:00
psg                 input/edf_data/3c1c5cf-PSG.edf
hypnogram     input/edf_data/3c1c5cf-Hypnogram.edf
Name: 0, dtype: object

## EDFファイルの読み込み

`edf`ファイルは医療時系列データの保存によく使用されるデータフォーマットになります。  
ロードするためには専用のライブラリが必要となります。

このノートブックでは、脳波などを解析する際に用いられるpythonライブラリである`NME`を使用します。  

----
❗❗ EDFファイルの扱いは専門的で、それを扱うMNEライブラリを詳しく理解するのは大変です。  
それよりも、生データからsklearnなどで学習可能なデータセットへ変換するためににはどのようなデータフレームを作成すれば良いかを意識してみると良いでしょう。  

----

`mne.io.read_raw_edf()`メソッドで`edf`ファイルを読み込むことができます。 

**患者一人データを読み込むとcsvで数万行になるため、メモリの対策が必要になります。** 

`NME`ではEDFをファイルを読み込む際に、`preload=False`を設定できます。  
データのメタ情報のみ読み込む機能であり、省メモリで済むようになります。

詳しいリファレンスは以下をご参考下さい。  

https://mne.tools/stable/generated/mne.io.read_raw_edf.html

In [32]:
psg_edf = mne.io.read_raw_edf(row["psg"], preload=False)

Extracting EDF parameters from /content/input/edf_data/3c1c5cf-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [33]:
# 読み込んだデータは、mne.io.edf.edf.RawEDFクラスのインスタンスになります
type(psg_edf)

mne.io.edf.edf.RawEDF

In [34]:
# infoでメタ情報を表示できます
psg_edf.info

<Info | 7 non-empty values
 bads: []
 ch_names: EEG Fpz-Cz, EEG Pz-Oz, EOG horizontal, Resp oro-nasal, EMG ...
 chs: 7 EEG
 custom_ref_applied: False
 highpass: 0.5 Hz
 lowpass: 100.0 Hz
 meas_date: 1989-11-13 16:35:00 UTC
 nchan: 7
 projs: []
 sfreq: 100.0 Hz
>

センサーチャンネルの名前を確認します

In [35]:
psg_edf.ch_names

['EEG Fpz-Cz',
 'EEG Pz-Oz',
 'EOG horizontal',
 'Resp oro-nasal',
 'EMG submental',
 'Temp rectal',
 'Event marker']

それぞれのチャンネルは睡眠ポリグラフからのセンサーチャンネルの情報になっています。  


センサーチャンネルについて簡単に説明します。  

- EEG： 脳波。すべての予測段階の判定に必要であり、意識水準と対応して変化する。
  - Fpz-Cz、Pz-Ozは頭に取り付けるセンサー箇所の箇所の電位差を表す
- EOG： 眼電図。 眼球運動を測定。
  - horizontalは水平方向に取り付けた電位差を表す
- EMG： オトガイ(顎下)筋電図。
- Resp oro-nasal：口鼻呼吸 (oroが口、nasalが鼻、respはrespirationの略で呼吸を表す)
- Temp rectal：直腸温 (原著に記載がないため、実際に直腸の温度を計測したかどうかは不明)
- Event marker：各時刻で起きたイベントの時刻

<img src="https://sas.ochis-net.jp/wp-content/uploads/2018/04/kensa.jpg" ><br />
出典：NPO法人ヘルスケアネットワーク


詳細についてはデータの公開先([Sleep-EDF Database Expanded](https://www.physionet.org/content/sleep-edfx/1.0.0/))をご参照下さい。

In [36]:
# to_data_frame()でpandasのDataFrameに変換できます
psg_df = psg_edf.to_data_frame()

In [37]:
psg_df

,time,EEG Fpz-Cz,EEG Pz-Oz,EOG horizontal,Resp oro-nasal,EMG submental,Temp rectal,Event marker
0,0.00,84.851770,-11.197558,-219.482051,8.800000e+07,3.234000,1.427420e+07,9.490000e+08
1,0.01,87.957998,13.197558,-215.531868,8.790847e+07,3.233153,1.428125e+07,9.499478e+08
2,0.02,96.862515,-3.908181,-209.112821,8.781710e+07,3.232296,1.428833e+07,9.509124e+08
3,0.03,106.181197,-2.644689,-204.175092,8.772580e+07,3.231429,1.429543e+07,9.518927e+08
4,0.04,102.557265,-25.095971,-198.249817,8.763449e+07,3.230552,1.430254e+07,9.528878e+08
...,...,...,...,...,...,...,...,...
7889995,78899.95,-0.776557,-3.422222,12.097436,6.878336e+07,3.304648,1.428788e+07,9.730920e+08
7889996,78899.96,2.950916,-1.284005,11.109890,6.845655e+07,3.304943,1.428799e+07,9.730803e+08
7889997,78899.97,-12.476679,-0.409280,-1.728205,6.811467e+07,3.305226,1.428810e+07,9.730652e+08
7889998,78899.98,-3.261538,-13.530159,6.172161,6.775786e+07,3.305496,1.428820e+07,9.730468e+08


生データを見て分かること

* アノテーションのカラムがないので紐付ける必要がある
* timeは計測の経過時間で、計測日時ではない。提出のフォーマットは計測日時なので変換する必要がある。
* Sampling frequencyは100Hzなので、0.00秒から始まっている。したがって、データの合計時間は(7890000/100)/3600=約22時間となる。  


計測時間の開始は`meas_date`で取得できます。  
これから100ミリ秒の間隔で時間を計測すると、16時から翌14時過ぎまで計測していることが分かります

In [38]:
psg_edf.info["meas_date"]

datetime.datetime(1989, 11, 13, 16, 35, tzinfo=datetime.timezone.utc)

In [39]:
meas_start = psg_edf.info["meas_date"]
meas_start = meas_start.replace(tzinfo=None)
# 100Hz
psg_df["meas_time"] = pd.date_range(start=meas_start, periods=len(psg_df), freq=pd.Timedelta(1 / 100, unit="s"))

In [40]:
psg_df

,time,EEG Fpz-Cz,EEG Pz-Oz,EOG horizontal,Resp oro-nasal,EMG submental,Temp rectal,Event marker,meas_time
0,0.00,84.851770,-11.197558,-219.482051,8.800000e+07,3.234000,1.427420e+07,9.490000e+08,1989-11-13 16:35:00.000
1,0.01,87.957998,13.197558,-215.531868,8.790847e+07,3.233153,1.428125e+07,9.499478e+08,1989-11-13 16:35:00.010
2,0.02,96.862515,-3.908181,-209.112821,8.781710e+07,3.232296,1.428833e+07,9.509124e+08,1989-11-13 16:35:00.020
3,0.03,106.181197,-2.644689,-204.175092,8.772580e+07,3.231429,1.429543e+07,9.518927e+08,1989-11-13 16:35:00.030
4,0.04,102.557265,-25.095971,-198.249817,8.763449e+07,3.230552,1.430254e+07,9.528878e+08,1989-11-13 16:35:00.040
...,...,...,...,...,...,...,...,...,...
7889995,78899.95,-0.776557,-3.422222,12.097436,6.878336e+07,3.304648,1.428788e+07,9.730920e+08,1989-11-14 14:29:59.950
7889996,78899.96,2.950916,-1.284005,11.109890,6.845655e+07,3.304943,1.428799e+07,9.730803e+08,1989-11-14 14:29:59.960
7889997,78899.97,-12.476679,-0.409280,-1.728205,6.811467e+07,3.305226,1.428810e+07,9.730652e+08,1989-11-14 14:29:59.970
7889998,78899.98,-3.261538,-13.530159,6.172161,6.775786e+07,3.305496,1.428820e+07,9.730468e+08,1989-11-14 14:29:59.980


## 脳波データをグラフ化してみる

1989-11-14の2時から2時05分までのEEG Fpz-Czを可視化してみる

In [41]:
temp = psg_df[(psg_df['meas_time'] >= pd.to_datetime('1989-11-14 2:00:00')) \
              & (psg_df['meas_time'] < pd.to_datetime('1989-11-14 2:05:00'))][["EEG Fpz-Cz", "meas_time"]]

In [42]:
temp

,EEG Fpz-Cz,meas_time
3390000,-50.269109,1989-11-14 02:00:00.000
3390001,-49.130159,1989-11-14 02:00:00.010
3390002,-49.026618,1989-11-14 02:00:00.020
3390003,-44.988523,1989-11-14 02:00:00.030
3390004,-45.195604,1989-11-14 02:00:00.040
...,...,...
3419995,-55.653236,1989-11-14 02:04:59.950
3419996,-34.323810,1989-11-14 02:04:59.960
3419997,-55.239072,1989-11-14 02:04:59.970
3419998,-48.094750,1989-11-14 02:04:59.980


In [43]:
px.line(temp, x="meas_time", y="EEG Fpz-Cz")

## アノテーションデータの読み込み

In [44]:
# Hypnogramはread_annotationsで読み込むことが可能です
annot = mne.read_annotations(row["hypnogram"])

In [45]:
# 同様にto_data_frameでdataframe化できます
annot_df = annot.to_data_frame()

In [46]:
annot_df

,onset,duration,description
0,1970-01-01 00:00:00,16290.0,Sleep stage W
1,1970-01-01 04:31:30,30.0,Sleep stage 1
2,1970-01-01 04:32:00,90.0,Sleep stage 2
3,1970-01-01 04:33:30,1890.0,Sleep stage W
4,1970-01-01 05:05:00,30.0,Sleep stage 1
...,...,...,...
132,1970-01-01 17:37:30,30.0,Sleep stage 1
133,1970-01-01 17:38:00,270.0,Sleep stage W
134,1970-01-01 17:42:30,30.0,Sleep stage 1
135,1970-01-01 17:43:00,15120.0,Sleep stage W


アノテーションは開始時間とその間隔で構成されています。 

カラムの内容を簡単に説明します
- onset：経過時間（年と日付は設定が必要なので正しくありません。この例では時間の経過のみが正しいものになります）
- duration：アノテーションの間隔時間
- description：睡眠段階のラベル

最初と最後にかなり長い`Sleep Stage W`があることが分かります

In [47]:
annot_df["description"].value_counts()

Sleep stage 1    47
Sleep stage 2    39
Sleep stage W    28
Sleep stage 3    12
Sleep stage R     8
Sleep stage ?     3
Name: description, dtype: int64

# 4.波形データとアノテーションデータの紐づけ

波形データとアノテーションデータは別々のデータ構造を持っているため、波形データにアノテーションを紐付ける作業が必要になります。

波形データを30秒毎に区切って1epochとして、`onset`と`duration`から睡眠段階を紐付けます。  
`mne`ライブラリを利用してこれを行いたいと思います。  

処理を簡単にするために、センサーは`EEG Fpz-Cz`のみを利用します(データを読み込む時に`EEG Fpz-Cz`を指定) 

最終的に以下のようなデータフレームを生成します

| 経過時間 | 測定日時                    | エポック | ラベル           | センサーデータ    |
|------|-------------------------|------|---------------|------------|
| 0.00 | 1990-03-20 20:20:00.000 | 0    | Sleep stage 2 | -17.200244 |
| 0.01 | 1990-03-20 20:20:00.010 | 0    | Sleep stage 2 | -16.910867 |
| 0.02 | 1990-03-20 20:20:00.020 | 0    | Sleep stage 2 | -16.910867 |
| 0.03 | 1990-03-20 20:20:00.030 | 0    | Sleep stage 2 | -16.910867 |
| 0.04 | 1990-03-20 20:20:00.040 | 0    | Sleep stage 2 | -17.875458 |



また、測定前後にかなり長い`Sleep Stage W`があるので。  
データ数を減らすため、適当に前後5時間で解析時間の切り捨てを行います  
どのくらいデータを切り捨てると良いか試行錯誤するのもいいでしょう。

<img src="https://i.gyazo.com/6bda14d1c7064748f212869d2ab37f45.png" />

In [48]:
# ラベル名をIDに置き換える
# Sleep stage 3とSleep stage 4を同じIDとして、AASMによる分類に変更する
RANDK_LABEL2ID = {
    'Movement time': -1,
    'Sleep stage ?': -1,
    'Sleep stage W': 0,
    'Sleep stage 1': 1,
    'Sleep stage 2': 2,
    'Sleep stage 3': 3,
    'Sleep stage 4': 3,
    'Sleep stage R': 4
}

# AASMによる分類
LABEL2ID = {
    'Movement time': -1,
    'Sleep stage ?': -1,
    'Sleep stage W': 0,
    'Sleep stage 1': 1,
    'Sleep stage 2': 2,
    'Sleep stage 3/4': 3,
    'Sleep stage R': 4
}
ID2LABEL = {v:k for k, v in LABEL2ID.items()}

In [49]:
# 再度データを読み込みます
# 簡単のためにEEG Fpz-Czのみ利用します
psg_edf = mne.io.read_raw_edf(row["psg"], include=["EEG Fpz-Cz"], verbose=False)
annot = mne.read_annotations(row["hypnogram"])

# 前後5時間を切り捨て（1時間(3600秒)*5）
# │------<切り捨て --------- 切り捨て>------│
truncate_start_point = 3600 * 5
truncate_end_point = (len(psg_edf)/100) - (3600 * 5)
# 切り捨て
annot.crop(truncate_start_point, truncate_end_point, verbose=False)

# annotationデータを紐づけます
psg_edf.set_annotations(annot, verbose=False, emit_warning=False)

# イベントが起きた経過時間30秒間隔を取得
events, _ = mne.events_from_annotations(psg_edf, event_id=RANDK_LABEL2ID, chunk_duration=30., verbose=False)

`events`のarray. 

- 1列目：イベントが記録された計測開始からの経過時間(10m秒)  
  - 最初が1800000[10m秒]=5時間後
- 3列目：アノテーションのID になっています

（今回の例では2列目は特に考慮する必要がありません）

In [50]:
events

array([[1800000,       0,       0],
       [1803000,       0,       0],
       [1806000,       0,       0],
       ...,
       [6081000,       0,       0],
       [6084000,       0,       1],
       [6087000,       0,       1]])

In [51]:
# 30秒毎に1epochとする
tmax = 30. - 1. / psg_edf.info['sfreq']

# eventsを引数にとってepochをインスタンス化
epoch = mne.Epochs(
    raw=psg_edf,
    events=events,
    event_id=LABEL2ID,
    tmin=0,
    tmax=tmax,
    baseline=None,
    verbose=False, 
    on_missing='ignore',
)

# subjectとnightの設定 
epoch.info["temp"] = {
    "id":row["id"],
    "subject_id":row["subject_id"],
    "night":row["night"],
    "truncate_start_point":truncate_start_point,
    "truncate_end_point":truncate_end_point
}

`mne.epochs.Epochs`クラスは波形データにラベルを紐づけたインスタンスになります

In [52]:
print(type(epoch))
epoch

<class 'mne.epochs.Epochs'>


Number of events,1430
Events,Movement time: 20Sleep stage 1: 122Sleep stage 2: 422Sleep stage 3/4: 27Sleep stage ?: 20Sleep stage R: 35Sleep stage W: 804
Time range,0.000 – 29.990 sec
Baseline,off


`RawEDF`クラスと同様にデータフレーム化できます。

データフレームを生成して確認します

In [53]:
epoch_df = epoch.to_data_frame(verbose=False)
epoch_df

,time,condition,epoch,EEG Fpz-Cz
0,0.00,Sleep stage W,0,-80.295971
1,0.01,Sleep stage W,0,-32.252991
2,0.02,Sleep stage W,0,-88.165079
3,0.03,Sleep stage W,0,15.893529
4,0.04,Sleep stage W,0,23.555556
...,...,...,...,...
4289995,29.95,Sleep stage 1,1429,7.092552
4289996,29.96,Sleep stage 1,1429,0.155311
4289997,29.97,Sleep stage 1,1429,6.574847
4289998,29.98,Sleep stage 1,1429,-6.678388


`epoch`のカラムが追加されていることが分かります。  
`epoch`は0.00〜29.99秒の30秒で、1つのepochとなっていることが分かります。  
`time`はsampling rateが100Hzなので0.00秒になっています。  

### 計測時間の設定

`epoch_df`のtimeカラムは0.00〜29.99秒のエポック内の秒数になっており連続した測定時間になっていません。  

submittionの形式と異なっています。  なので、計測時間から連続した時間を振ります。


データにはMeasurement date(測定日)の情報がありましたが、切り捨てをしたため測定日を計算し直す必要があります。  
具体的には、切り捨てが発生した`truncate_start_point`を`start`として測定日に設定します。  
終了時刻は、データ点の長さ分を100Hzの時間単位で計算すれば決まります。

In [54]:
# 測定日を切り捨てが発生したポイントまでスライド
new_meas_date = epoch.info["meas_date"].replace(tzinfo=None) \
+ datetime.timedelta(seconds=epoch.info["temp"]["truncate_start_point"])

# 切り捨て終了ポイントをstart連続した時間を作成
epoch_df["meas_time"] = pd.date_range(start=new_meas_date, periods=len(epoch_df), freq=pd.Timedelta(1 / 100, unit="s"))

In [55]:
epoch_df

,time,condition,epoch,EEG Fpz-Cz,meas_time
0,0.00,Sleep stage W,0,-80.295971,1989-11-13 21:35:00.000
1,0.01,Sleep stage W,0,-32.252991,1989-11-13 21:35:00.010
2,0.02,Sleep stage W,0,-88.165079,1989-11-13 21:35:00.020
3,0.03,Sleep stage W,0,15.893529,1989-11-13 21:35:00.030
4,0.04,Sleep stage W,0,23.555556,1989-11-13 21:35:00.040
...,...,...,...,...,...
4289995,29.95,Sleep stage 1,1429,7.092552,1989-11-14 09:29:59.950
4289996,29.96,Sleep stage 1,1429,0.155311,1989-11-14 09:29:59.960
4289997,29.97,Sleep stage 1,1429,6.574847,1989-11-14 09:29:59.970
4289998,29.98,Sleep stage 1,1429,-6.678388,1989-11-14 09:29:59.980


21時から翌の9時までの睡眠の波形データとアノテーションが紐付いたデータが作成されました。  
csvデータとして扱いたい場合、このdataframeを保存しておくとよいかもしれません。  


epochを受け取って、meas_dateを計算する部分を関数化しておきます。

In [56]:
def epoch_to_df(epoch:mne.epochs.Epochs) -> pd.DataFrame:
    truncate_start_point = epoch.info["temp"]["truncate_start_point"]
    
    df = epoch.to_data_frame(verbose=False)
    
    new_meas_date = epoch.info["meas_date"].replace(tzinfo=None) + datetime.timedelta(seconds=truncate_start_point)
    
    df["meas_time"] = pd.date_range(start=new_meas_date, periods=len(df), freq=pd.Timedelta(1 / 100, unit="s"))
    
    return df

In [57]:
epoch_to_df(epoch)

,time,condition,epoch,EEG Fpz-Cz,meas_time
0,0.00,Sleep stage W,0,-80.295971,1989-11-13 21:35:00.000
1,0.01,Sleep stage W,0,-32.252991,1989-11-13 21:35:00.010
2,0.02,Sleep stage W,0,-88.165079,1989-11-13 21:35:00.020
3,0.03,Sleep stage W,0,15.893529,1989-11-13 21:35:00.030
4,0.04,Sleep stage W,0,23.555556,1989-11-13 21:35:00.040
...,...,...,...,...,...
4289995,29.95,Sleep stage 1,1429,7.092552,1989-11-14 09:29:59.950
4289996,29.96,Sleep stage 1,1429,0.155311,1989-11-14 09:29:59.960
4289997,29.97,Sleep stage 1,1429,6.574847,1989-11-14 09:29:59.970
4289998,29.98,Sleep stage 1,1429,-6.678388,1989-11-14 09:29:59.980


ラベル、計測時間、脳波が紐付いたデータフレームが生成できました。  
**30秒毎に特徴量を生成することで学習可能なデータセットになります。**

### submission形式への変換
`epoch`毎にグループ化して`time`の最小値を取ることでsubmissionする形式に変換できます

In [58]:
label_df = epoch_df.loc[epoch_df.groupby("epoch")["time"].idxmin()][["meas_time"]].reset_index(drop=True)
label_df["condition"] = "Sleep stage W"
label_df["id"] = epoch.info["temp"]["id"]
label_df

,meas_time,condition,id
0,1989-11-13 21:35:00,Sleep stage W,3c1c5cf
1,1989-11-13 21:35:30,Sleep stage W,3c1c5cf
2,1989-11-13 21:36:00,Sleep stage W,3c1c5cf
3,1989-11-13 21:36:30,Sleep stage W,3c1c5cf
4,1989-11-13 21:37:00,Sleep stage W,3c1c5cf
...,...,...,...
1425,1989-11-14 09:27:30,Sleep stage W,3c1c5cf
1426,1989-11-14 09:28:00,Sleep stage W,3c1c5cf
1427,1989-11-14 09:28:30,Sleep stage W,3c1c5cf
1428,1989-11-14 09:29:00,Sleep stage W,3c1c5cf


関数化しておきます

In [59]:
def epoch_to_sub_df(epoch_df:pd.DataFrame, id, is_train:bool) -> pd.DataFrame:
    cols = ["id", "meas_time"]
    if is_train:
        # 訓練セットの場合はラベルを追加
        cols.append("condition")
    
    label_df = epoch_df.loc[epoch_df.groupby("epoch")["time"].idxmin()].reset_index(drop=True)
    label_df["id"] = id
    
    return label_df[cols]

In [60]:
epoch_to_sub_df(epoch_df, epoch.info["temp"]["id"], is_train=True)

,id,meas_time,condition
0,3c1c5cf,1989-11-13 21:35:00,Sleep stage W
1,3c1c5cf,1989-11-13 21:35:30,Sleep stage W
2,3c1c5cf,1989-11-13 21:36:00,Sleep stage W
3,3c1c5cf,1989-11-13 21:36:30,Sleep stage W
4,3c1c5cf,1989-11-13 21:37:00,Sleep stage W
...,...,...,...
1425,3c1c5cf,1989-11-14 09:27:30,Sleep stage W
1426,3c1c5cf,1989-11-14 09:28:00,Sleep stage W
1427,3c1c5cf,1989-11-14 09:28:30,Sleep stage W
1428,3c1c5cf,1989-11-14 09:29:00,Sleep stage 1


# 5.テストデータに対する前処理

テストデータにはアノテーションファイルがないので`epoch`の生成に必要な`events`の行列を、先程の例で作成することはできません。  

したがって、自前でイベントがおきた経過時間を求めます。

また、テストセットの脳波の測定開始時刻と評価対象の時刻は異なっています。  
したがって、評価対象の時刻に合わせて脳波データを切り捨てる必要があります。


In [61]:
test_row = test_record_df.iloc[0]
psg_edf = mne.io.read_raw_edf(test_row["psg"], include=["EEG Fpz-Cz"], verbose=False)

In [62]:
psg_edf.info["meas_date"]

datetime.datetime(1989, 11, 20, 15, 16, tzinfo=datetime.timezone.utc)

In [63]:
sample_submission_df[sample_submission_df["id"]==test_row["id"]]

,id,meas_time,condition
0,53c1555,1989-11-20 23:19:30,Sleep stage W
1,53c1555,1989-11-20 23:20:00,Sleep stage W
2,53c1555,1989-11-20 23:20:30,Sleep stage W
3,53c1555,1989-11-20 23:21:00,Sleep stage W
4,53c1555,1989-11-20 23:21:30,Sleep stage W
...,...,...,...
1058,53c1555,1989-11-21 08:08:30,Sleep stage W
1059,53c1555,1989-11-21 08:09:00,Sleep stage W
1060,53c1555,1989-11-21 08:09:30,Sleep stage W
1061,53c1555,1989-11-21 08:10:00,Sleep stage W


eventの開始時刻. 

23:19:30 - 15:16:00 = 29010.00秒

<img src="https://i.gyazo.com/a7701b7b0e67301e4caddde503e8e44f.png" />

In [64]:
# 測定の開始時間をedfのmeas_dateから取得します
start_psg_date = psg_edf.info["meas_date"]
# 計算のためにtimezoneを消去します
start_psg_date = start_psg_date.replace(tzinfo=None)

# sample submissionから評価される時間レンジを取得します
test_start_time = sample_submission_df[sample_submission_df["id"]==test_row["id"]]["meas_time"].min()
test_end_time = sample_submission_df[sample_submission_df["id"]==test_row["id"]]["meas_time"].max()

# psgの計測開始、評価の対象の開始、評価の対象の終了
print(f"psg start: {start_psg_date},  test start: {test_start_time}, test end: {test_end_time}")

# psgの計測時間から評価の対象の開始と終了を経過時間(秒)になおす
truncate_start_point = int((test_start_time - start_psg_date).total_seconds())
truncate_end_point = int((test_end_time - start_psg_date).total_seconds())+30
print(f"event start sencond: {truncate_start_point}, event end second: {truncate_end_point} ")

# 30秒毎にデータ点を生成
event_range = list(range(truncate_start_point, truncate_end_point, 30))
events = np.zeros((len(event_range), 3), dtype=int)
events[:, 0] = event_range

# 秒を10m秒に変換する
events = events * 100

psg start: 1989-11-20 15:16:00,  test start: 1989-11-20 23:19:30, test end: 1989-11-21 08:10:30
event start sencond: 29010, event end second: 60900 


In [65]:
events

array([[2901000,       0,       0],
       [2904000,       0,       0],
       [2907000,       0,       0],
       ...,
       [6081000,       0,       0],
       [6084000,       0,       0],
       [6087000,       0,       0]])

In [66]:
tmax = 30. - 1. / psg_edf.info['sfreq']

# 「Sleep stage W」はダミー
epoch = mne.Epochs(
    raw=psg_edf,
    events=events,
    event_id={'Sleep stage W': 0},
    tmin=0,
    tmax=tmax,
    baseline=None,
    verbose=False)

In [67]:
epoch.to_data_frame()

Loading data for 1063 events and 3000 original time points ...
0 bad epochs dropped


,time,condition,epoch,EEG Fpz-Cz
0,0.00,Sleep stage W,0,-10.738462
1,0.01,Sleep stage W,0,-13.220513
2,0.02,Sleep stage W,0,-31.481319
3,0.03,Sleep stage W,0,-10.915751
4,0.04,Sleep stage W,0,-39.902564
...,...,...,...,...
3188995,29.95,Sleep stage W,1062,-37.065934
3188996,29.96,Sleep stage W,1062,-34.761172
3188997,29.97,Sleep stage W,1062,-37.331868
3188998,29.98,Sleep stage W,1062,-22.882784


# 6.前処理関数

患者データを受け取り、`Epoch`を生成する関数  
`Epoch`生成後は、`epoch_to_df`を通せばDataFrameを生成できる

In [68]:
def read_and_set_annoation(record_df:pd.DataFrame, include=None, is_test=False) -> List[mne.epochs.Epochs]:
    whole_epoch_data = []

    for row_id, row in tqdm(record_df.iterrows(), total=len(record_df)):        
        # PSGファイルとHypnogram(アノテーションファイルを読み込む)
        psg_edf = mne.io.read_raw_edf(row["psg"], include=include, verbose=False)
        
        if not is_test:
            # 訓練データの場合
            annot = mne.read_annotations(row["hypnogram"])

            # 切り捨て
            truncate_start_point = 3600 * 5
            truncate_end_point = (len(psg_edf)/100) - (3600 *5)
            annot.crop(truncate_start_point, truncate_end_point, verbose=False)

            # アノテーションデータの切り捨て
            psg_edf.set_annotations(annot, emit_warning=False)
            events, _ = mne.events_from_annotations(psg_edf, event_id=RANDK_LABEL2ID, chunk_duration=30., verbose=False)
            
            event_id = LABEL2ID
        else:
            # テストデータの場合
            start_psg_date = psg_edf.info["meas_date"]
            start_psg_date = start_psg_date.replace(tzinfo=None)

            test_start_time = sample_submission_df[sample_submission_df["id"]==row["id"]]["meas_time"].min()
            test_end_time = sample_submission_df[sample_submission_df["id"]==row["id"]]["meas_time"].max()
            
            truncate_start_point = int((test_start_time - start_psg_date).total_seconds())
            truncate_end_point = int((test_end_time- start_psg_date).total_seconds())+30
            
            event_range = list(range(truncate_start_point, truncate_end_point, 30))
            events = np.zeros((len(event_range), 3), dtype=int)
            events[:, 0] = event_range
            events = events * 100
            
            event_id = {'Sleep stage W': 0}
            
    
        # 30秒毎に1epochとする
        tmax = 30. - 1. / psg_edf.info['sfreq']
        epoch = mne.Epochs(raw=psg_edf, events=events, event_id=event_id, tmin=0, tmax=tmax, baseline=None, verbose=False, on_missing='ignore')
        
        # 途中でデータが落ちてないかチェック
        assert len(epoch.events) * 30 == truncate_end_point - truncate_start_point
        
        # メタデータを追加
        epoch.info["temp"] = {
            "id":row["id"],
            "subject_id":row["subject_id"],
            "night":row["night"],
            "age":row["age"],
            "sex":row["sex"],
            "truncate_start_point":truncate_start_point
        }

        whole_epoch_data.append(epoch)

    return whole_epoch_data 

In [69]:
# 処理を簡単にするためにEEG Fpz-Czのみ読み込みます
# trainをすべて処理するには少し時間がかかるため、ここでは半分ほどの50を利用することにします
train_record_subset_df = train_record_df.sample(n=50).reset_index(drop=True)

In [70]:
# epochインスタンスを生成
train_subset_epoch = read_and_set_annoation(train_record_subset_df, include=["EEG Fpz-Cz"], is_test=False)
test_whole_epoch = read_and_set_annoation(test_record_df, include=["EEG Fpz-Cz"], is_test=True)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

In [71]:
train_subset_epoch[:2]

[<Epochs |  1510 events (good & bad), 0 - 29.99 sec, baseline off, ~7 kB, data not loaded,
  'Movement time': 0
  'Sleep stage ?': 0
  'Sleep stage W': 885
  'Sleep stage 1': 78
  'Sleep stage 2': 419
  'Sleep stage 3/4': 104
  'Sleep stage R': 24>,
 <Epochs |  1606 events (good & bad), 0 - 29.99 sec, baseline off, ~7 kB, data not loaded,
  'Movement time': 2
  'Sleep stage ?': 2
  'Sleep stage W': 581
  'Sleep stage 1': 113
  'Sleep stage 2': 616
  'Sleep stage 3/4': 114
  'Sleep stage R': 180>]

# 7.簡単な特徴量エンジニアリング

30秒(epoch)ごとに特徴量を生成する必要があります。  
ここでは簡単にepoch毎にEEGの平均をとってみます。

<img src="https://i.gyazo.com/b21ffb0df3d3d1aec219d908cd084cfc.png" />

In [72]:
# epochインスタンスからepochのDataFrameを生成
epoch_df = epoch_to_df(train_subset_epoch[0])
# submit形式のデータフレーム生成
sub_df = epoch_to_sub_df(epoch_df, train_subset_epoch[0].info["temp"]["id"], is_train=True)

In [73]:
epoch_df

,time,condition,epoch,EEG Fpz-Cz,meas_time
0,0.00,Sleep stage W,0,8.952381,1990-05-28 21:00:00.000
1,0.01,Sleep stage W,0,6.840537,1990-05-28 21:00:00.010
2,0.02,Sleep stage W,0,3.994139,1990-05-28 21:00:00.020
3,0.03,Sleep stage W,0,7.299634,1990-05-28 21:00:00.030
4,0.04,Sleep stage W,0,11.064225,1990-05-28 21:00:00.040
...,...,...,...,...,...
4529995,29.95,Sleep stage W,1509,-3.994139,1990-05-29 09:34:59.950
4529996,29.96,Sleep stage W,1509,-0.688645,1990-05-29 09:34:59.960
4529997,29.97,Sleep stage W,1509,-0.413187,1990-05-29 09:34:59.970
4529998,29.98,Sleep stage W,1509,-12.257875,1990-05-29 09:34:59.980


In [74]:
sub_df

,id,meas_time,condition
0,88c1130,1990-05-28 21:00:00,Sleep stage W
1,88c1130,1990-05-28 21:00:30,Sleep stage W
2,88c1130,1990-05-28 21:01:00,Sleep stage W
3,88c1130,1990-05-28 21:01:30,Sleep stage W
4,88c1130,1990-05-28 21:02:00,Sleep stage W
...,...,...,...
1505,88c1130,1990-05-29 09:32:30,Sleep stage W
1506,88c1130,1990-05-29 09:33:00,Sleep stage W
1507,88c1130,1990-05-29 09:33:30,Sleep stage W
1508,88c1130,1990-05-29 09:34:00,Sleep stage W


In [75]:
feature_df = epoch_df.groupby("epoch")["EEG Fpz-Cz"].mean()
feature_df

epoch
0       0.000092
1      -0.029780
2      -0.008203
3       0.130169
4      -0.374990
          ...   
1505    0.291618
1506   -0.570198
1507    0.245647
1508   -0.067334
1509   -0.251309
Name: EEG Fpz-Cz, Length: 1510, dtype: float64

In [76]:
#@title
sub_df

,id,meas_time,condition
0,88c1130,1990-05-28 21:00:00,Sleep stage W
1,88c1130,1990-05-28 21:00:30,Sleep stage W
2,88c1130,1990-05-28 21:01:00,Sleep stage W
3,88c1130,1990-05-28 21:01:30,Sleep stage W
4,88c1130,1990-05-28 21:02:00,Sleep stage W
...,...,...,...
1505,88c1130,1990-05-29 09:32:30,Sleep stage W
1506,88c1130,1990-05-29 09:33:00,Sleep stage W
1507,88c1130,1990-05-29 09:33:30,Sleep stage W
1508,88c1130,1990-05-29 09:34:00,Sleep stage W


In [77]:
pd.concat([sub_df, feature_df], axis=1)

,id,meas_time,condition,EEG Fpz-Cz
0,88c1130,1990-05-28 21:00:00,Sleep stage W,0.000092
1,88c1130,1990-05-28 21:00:30,Sleep stage W,-0.029780
2,88c1130,1990-05-28 21:01:00,Sleep stage W,-0.008203
3,88c1130,1990-05-28 21:01:30,Sleep stage W,0.130169
4,88c1130,1990-05-28 21:02:00,Sleep stage W,-0.374990
...,...,...,...,...
1505,88c1130,1990-05-29 09:32:30,Sleep stage W,0.291618
1506,88c1130,1990-05-29 09:33:00,Sleep stage W,-0.570198
1507,88c1130,1990-05-29 09:33:30,Sleep stage W,0.245647
1508,88c1130,1990-05-29 09:34:00,Sleep stage W,-0.067334


trainとtestでループして実行する

In [78]:
train_df = []
for epoch in tqdm(train_subset_epoch):
    # 波形をdataframe化
    epoch_df = epoch_to_df(epoch)
    # submit形式のデータフレーム生成
    sub_df = epoch_to_sub_df(epoch_df, epoch.info["temp"]["id"], is_train=True)

    # 平均を取る
    feature_df = epoch_df.groupby("epoch")["EEG Fpz-Cz"].mean()
    
    _df = pd.concat([sub_df, feature_df], axis=1)
    
    # 必要ないラベルがある場合は除外する
    _df = _df[~_df["condition"].isin(["Sleep stage ?", "Movement time"])]
    
    train_df.append(_df)
    
train_df = pd.concat(train_df).reset_index(drop=True)

  0%|          | 0/50 [00:00<?, ?it/s]

In [79]:
test_df = []
for epoch in tqdm(test_whole_epoch):
    # 波形をdataframe化
    epoch_df = epoch_to_df(epoch)
    # submit形式のデータフレーム生成
    sub_df = epoch_to_sub_df(epoch_df, epoch.info["temp"]["id"], is_train=False)

    # 平均を取る
    feature_df = epoch_df.groupby("epoch")["EEG Fpz-Cz"].mean()
    
    _df = pd.concat([sub_df, feature_df], axis=1)

    # testには「Sleep stage ?」と「Movement time」が存在しないので特別な処理はしない
    
    test_df.append(pd.concat([sub_df, feature_df], axis=1))
    
test_df = pd.concat(test_df)

  0%|          | 0/45 [00:00<?, ?it/s]

In [80]:
train_df

,id,meas_time,condition,EEG Fpz-Cz
0,88c1130,1990-05-28 21:00:00,Sleep stage W,0.000092
1,88c1130,1990-05-28 21:00:30,Sleep stage W,-0.029780
2,88c1130,1990-05-28 21:01:00,Sleep stage W,-0.008203
3,88c1130,1990-05-28 21:01:30,Sleep stage W,0.130169
4,88c1130,1990-05-28 21:02:00,Sleep stage W,-0.374990
...,...,...,...,...
75233,0bc4656,1990-07-14 10:17:30,Sleep stage W,0.682716
75234,0bc4656,1990-07-14 10:18:00,Sleep stage W,0.940584
75235,0bc4656,1990-07-14 10:18:30,Sleep stage W,0.930184
75236,0bc4656,1990-07-14 10:19:00,Sleep stage W,0.535952


In [81]:
test_df

,id,meas_time,EEG Fpz-Cz
0,53c1555,1989-11-20 23:19:30,-0.970171
1,53c1555,1989-11-20 23:20:00,0.065435
2,53c1555,1989-11-20 23:20:30,-0.538413
3,53c1555,1989-11-20 23:21:00,-0.601439
4,53c1555,1989-11-20 23:21:30,-0.305632
...,...,...,...
906,9b444bb,1989-04-12 07:32:30,0.327378
907,9b444bb,1989-04-12 07:33:00,-0.082026
908,9b444bb,1989-04-12 07:33:30,0.943103
909,9b444bb,1989-04-12 07:34:00,-0.700759


# 8.モデルの作成

## 検証セットの作成
trainとtestで被験者の重なりがないので、検証セットでも同様の状態を模倣します。  
具体的にはtrainから約20％の被験者を検証セットに割り当てます。


In [82]:
# 20％の被験者を選ぶ
np.random.seed(42)
val_size = int(train_record_df["subject_id"].nunique() * 0.20)
train_all_subjects = train_record_df["subject_id"].unique()
np.random.shuffle(train_all_subjects)

val_subjects = train_all_subjects[:val_size]
val_ids = train_record_df[train_record_df["subject_id"].isin(val_subjects)]["id"]

In [83]:
# 検証セットに選ばれた被験者
val_ids

10     738f8f4
11     75ab040
24     0aa8d07
25     1a3aa86
26     315edab
27     5070dd2
38     4decc81
39     e6ffc1c
52     77bf391
53     e3e504e
62     57bc285
63     60e9352
64     f7eacee
65     89e31c7
82     1a8be91
83     7b874dd
86     6c943b6
87     169c666
96     6bd6b4c
97     6ade5cb
102    e8d8e03
103    0691b9d
Name: id, dtype: object

In [84]:
# 検証セットを作成します
trn_df = train_df[~train_df["id"].isin(val_ids)]
val_df = train_df[train_df["id"].isin(val_ids)]

In [85]:
trn_df

,id,meas_time,condition,EEG Fpz-Cz
0,88c1130,1990-05-28 21:00:00,Sleep stage W,0.000092
1,88c1130,1990-05-28 21:00:30,Sleep stage W,-0.029780
2,88c1130,1990-05-28 21:01:00,Sleep stage W,-0.008203
3,88c1130,1990-05-28 21:01:30,Sleep stage W,0.130169
4,88c1130,1990-05-28 21:02:00,Sleep stage W,-0.374990
...,...,...,...,...
75233,0bc4656,1990-07-14 10:17:30,Sleep stage W,0.682716
75234,0bc4656,1990-07-14 10:18:00,Sleep stage W,0.940584
75235,0bc4656,1990-07-14 10:18:30,Sleep stage W,0.930184
75236,0bc4656,1990-07-14 10:19:00,Sleep stage W,0.535952


In [86]:
val_df

,id,meas_time,condition,EEG Fpz-Cz
15049,1a8be91,1990-03-12 20:35:00,Sleep stage W,-0.614742
15050,1a8be91,1990-03-12 20:35:30,Sleep stage W,-0.276680
15051,1a8be91,1990-03-12 20:36:00,Sleep stage W,-0.187676
15052,1a8be91,1990-03-12 20:36:30,Sleep stage W,-0.694429
15053,1a8be91,1990-03-12 20:37:00,Sleep stage W,-0.506846
...,...,...,...,...
69409,e6ffc1c,1989-05-03 10:02:30,Sleep stage W,0.026519
69410,e6ffc1c,1989-05-03 10:03:00,Sleep stage W,0.320859
69411,e6ffc1c,1989-05-03 10:03:30,Sleep stage W,-0.345659
69412,e6ffc1c,1989-05-03 10:04:00,Sleep stage W,0.384178


## 学習
ベースラインとしてランダムフォレストを利用します

In [87]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(trn_df.iloc[:, 3:], trn_df["condition"])

RandomForestClassifier(random_state=42)

## 精度の検証

In [88]:
# 検証セットの予測
val_preds = model.predict(val_df.iloc[:, 3:])

In [89]:
score = accuracy_score(val_df["condition"], val_preds)
print(score)

0.31278043132146477


In [90]:
print(classification_report(val_df["condition"], val_preds,))

                 precision    recall  f1-score   support

  Sleep stage 1       0.11      0.08      0.10      1615
  Sleep stage 2       0.29      0.31      0.30      3905
Sleep stage 3/4       0.07      0.05      0.06       898
  Sleep stage R       0.11      0.11      0.11      1464
  Sleep stage W       0.44      0.47      0.45      5936

       accuracy                           0.31     13818
      macro avg       0.20      0.20      0.20     13818
   weighted avg       0.30      0.31      0.31     13818



# 9.テストセットの予測

In [91]:
test_preds = model.predict(test_df.iloc[:, 2:])

In [92]:
# sample_submissionにセットしてラベル名に変更
sample_submission_df["condition"] = test_preds

In [93]:
sample_submission_df

,id,meas_time,condition
0,53c1555,1989-11-20 23:19:30,Sleep stage W
1,53c1555,1989-11-20 23:20:00,Sleep stage 1
2,53c1555,1989-11-20 23:20:30,Sleep stage W
3,53c1555,1989-11-20 23:21:00,Sleep stage W
4,53c1555,1989-11-20 23:21:30,Sleep stage W
...,...,...,...
52291,9b444bb,1989-04-12 07:32:30,Sleep stage W
52292,9b444bb,1989-04-12 07:33:00,Sleep stage 3/4
52293,9b444bb,1989-04-12 07:33:30,Sleep stage R
52294,9b444bb,1989-04-12 07:34:00,Sleep stage W


In [94]:
sample_submission_df["condition"].value_counts()

Sleep stage W      26188
Sleep stage 2      14073
Sleep stage R       5144
Sleep stage 1       4322
Sleep stage 3/4     2569
Name: condition, dtype: int64

In [95]:
sample_submission_df.to_csv("submission.csv", index=False)

これを提出するとスコアは0.27程度になります。  
ひとまず提出までたどり着きました。
お疲れ様でした。🍵    


データ分析は今回の様に泥臭い前処理が往々にして存在します。  
予測出すまでに辛い場合は、ひとまず学習して提出までやりきることも大切です。  

次回はドメイン知識を使った特徴量エンジニアリングを実施してスコアアップを目指します。